<h1><center>The rising use case of LLM: Structuring untructured data</center></h1>

## Demo Start

In [1]:
import os
import json
import time
from pathlib import Path

import pandas as pd
from langchain.output_parsers import PydanticOutputParser
from langchain_mistralai.chat_models import ChatMistralAI
from dotenv import load_dotenv

from core import run
from prompt import DEFAULT_BASE_PROMPT, create_prompt
from schemas import Recipe

In [2]:
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

In [3]:
path_to_data = Path(os.getcwd()) / "data" / "input"
df = pd.read_json(path_to_data / "recipes_v1.json")
df.head()

,title,date,tags,introduction,ingredients,direction,output
0,Creamy Mashed Potatoes,2021-03-12,"[potato, side, cheesefare]",![Creamy Mashed Potatoes](/pix/creamy-mashed-p...,The quantities here are for about four adult p...,1. Peel and cut the potatoes into medium sized...,"{'title': 'Creamy Mashed Potatoes', 'ingredien..."
1,Red Sauce (Ragu all'Italiana),2021-03-16,"[italian, sauce]",My great-grandma's red sauce. All purpose: goo...,- 1/3 lb salt pork - 2 lb chuck roast - 3 Clov...,"1. Mince the garlic, peel the carrot, peel and...","{'title': 'Red Sauce (Ragu all'Italiana)', 'in..."
2,Turmeric Flatbread,2022-08-09,"[bread, turkish]",A great companion to Turkish Red Lentil Soup. ...,- 1 cup Wheat Flour (white or whole) - 1/2 Tbs...,"1. Combine flour, turmeric, salt and baking po...","{'title': 'Turmeric Flatbread', 'ingredients':..."
3,Zurich-Style Meat Saute,2023-01-06,"[beef, cream, swiss, quick]","Originally called ""Züri Gschnätzlets"" (Zurich ...","- 600g\tBeef, finely sliced - 250g\tMushrooms,...",1. Fry the meat in butter at high heat until i...,"{'title': 'Zurich-Style Meat Saute', 'ingredie..."
4,Tuscan Style Pork Roast,2021-03-10,"[italian, pork, roast]",![tuscan-style-pork-roast](/pix/tuscan-style-p...,- 1 pork Roast - 2-3 Tbsp fresh rosemary - 8 c...,1.\tPreheat oven to 275°F (135°C) 2.\tButterfl...,"{'title': 'Tuscan Style Pork Roast', 'ingredie..."


In [4]:
model_name = "open-mixtral-8x7b"
#model_name = "open-mixtral-8x22b"
temperature = 0
llm = ChatMistralAI(api_key=MISTRAL_API_KEY, model_name=model_name, temperature=temperature)
parser = PydanticOutputParser(pydantic_object=Recipe)
prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["ingredients"][0], df["direction"][0])
prompt

[HumanMessage(content='\nWhat are the ingredients and their associated quantities as well as the steps to make the recipe described by the following The quantities here are for about four adult portions. If you are planning on eating this as a side dish, it might be more like 6-8 portions. * 1kg potatoes * 200ml milk* * 200ml mayonnaise* * ~100g cheese * Garlic powder * 12-16 strips of bacon * Butter * 3-4 green onions * Black pepper * Salt  *You can play with the proportions depending on how creamy or dry you want the mashed potatoes to be. and 1. Peel and cut the potatoes into medium sized pieces. 2. Put the potatoes in a pot with some water so that it covers the potatoes and   boil them for about 20-30 minutes, or until the potatoes are soft. 3. About ten minutes before removing the potatoes from the boiling water, cut   the bacon into little pieces and fry it. 4. Warm up the milk and mayonnaise. 5. Shred the cheese. 6. When the potatoes are done, remove all water from the pot, add 

In [ ]:
example = await run(llm, prompt, parser)
example

## End of Demo

In [5]:
print(prompt[0].content)


What are the ingredients and their associated quantities as well as the steps to make the recipe described by the following The quantities here are for about four adult portions. If you are planning on eating this as a side dish, it might be more like 6-8 portions. * 1kg potatoes * 200ml milk* * 200ml mayonnaise* * ~100g cheese * Garlic powder * 12-16 strips of bacon * Butter * 3-4 green onions * Black pepper * Salt  *You can play with the proportions depending on how creamy or dry you want the mashed potatoes to be. and 1. Peel and cut the potatoes into medium sized pieces. 2. Put the potatoes in a pot with some water so that it covers the potatoes and   boil them for about 20-30 minutes, or until the potatoes are soft. 3. About ten minutes before removing the potatoes from the boiling water, cut   the bacon into little pieces and fry it. 4. Warm up the milk and mayonnaise. 5. Shred the cheese. 6. When the potatoes are done, remove all water from the pot, add the warm milk   and mayo

## Trial and run on the entire dataset

In [6]:
raw_outputs = []
outputs = []

In [7]:
#df_sample = df.sample(10)

In [8]:
# df_sample= df_sample.reset_index(drop=True)
# for i in range(df_sample.shape[0]):
  
#     prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df_sample["ingredients"][i], df_sample["direction"][i])
#     recipe = await run(llm, prompt, parser)
#     raw_outputs.append(recipe)
    
#     output_map = {
#         "recipe_id":i,
#         "original_title":df_sample["title"][i],
#     }

#     if isinstance(recipe, Recipe):
#         output =  output_map | recipe.dict() 
#         outputs.append(output)
#     else:
#         output_map["raw_llm_output"] = recipe
#         outputs.append(output_map)

#     time.sleep(1)
   
        

In [7]:

for i in range(df.shape[0]):
  
    prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["ingredients"][i], df["direction"][i])
    recipe = await run(llm, prompt, parser)
    raw_outputs.append(recipe)
    
    output_map = {
        "recipe_id":i,
        "original_title":df["title"][i],
    }

    if isinstance(recipe, Recipe):
        output =  output_map | recipe.dict() 
        outputs.append(output)
    else:
        output_map["raw_llm_output"] = recipe
        outputs.append(output_map)

    time.sleep(1)

Error in parsing Invalid json output: Here is the JSON instance for the recipe:

<pre><code>{
  "name": "Spicy Mayonnaise with Sesame Oil",
  "serving_size": 4,
  "ingredients": [
    {
      "id": 1,
      "name": "mayonnaise",
      "quantity": 1,
      "unit": "cup"
    },
    {
      "id": 2,
      "name": "chili sauce",
      "quantity": 0.5,
      "unit": "cup"
    },
    {
      "id": 3,
      "name": "sesame oil",
      "quantity": 1,
      "unit": "tablespoon"
    }
  ],
  "steps": [
    {
      "number": 1,
      "description": "Add 2 parts mayonnaise to 1 part chili sauce in a bowl.",
      "preparation_time": 5,
      "used_ingredients": [1, 2]
    },
    {
      "number": 2,
      "description": "Add 1 tbsp of sesame oil for every cup of mayonnaise you use.",
      "preparation_time": 5,
      "used_ingredients": [3]
    },
    {
      "number": 3,
      "description": "Mix all of the ingredients together, and taste test it for more chili sauce or mayonnaise.",
      "prep

Retrying langchain_mistralai.chat_models.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ReadTimeout: .


Error in parsing Failed to parse Recipe from completion {"name": "Torrijas", "serving_size": 16, "ingredients": [{"id": 1, "name": "Dried bread (or Torrijas bread)", "quantity": null, "unit": null}, {"id": 2, "name": "Milk", "quantity": 2, "unit": "cups"}, {"id": 3, "name": "Eggs", "quantity": 2, "unit": null}, {"id": 4, "name": "Cinnamon", "quantity": null, "unit": "teaspoon"}, {"id": 5, "name": "Grated lemon rind", "quantity": null, "unit": null}, {"id": 6, "name": "Sugar", "quantity": null, "unit": "tablespoon"}, {"id": 7, "name": "Olive oil", "quantity": null, "unit": "pan (half)"}], "steps": [{"number": 1, "description": "Mull the milk with cinnamon and a bit of sugar (not too much, the milk is already sweet). Add the lemon rind if you want.", "preparation_time": 5, "cooking_time": null, "waiting_time": null, "used_ingredients": [{"id": 2}, {"id": 4}, {"id": 5}, {"id": 6}]}, {"number": 2, "description": "Cool down the milk.", "preparation_time": null, "cooking_time": null, "waitin

Retrying langchain_mistralai.chat_models.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ReadTimeout: .


Error in parsing Failed to parse Recipe from completion {"name": "Anglerfish with Butter and Herbs", "serving_size": 1, "ingredients": [{"id": 1, "name": "Anglerfish", "quantity": null, "unit": null}, {"id": 2, "name": "Butter", "quantity": null, "unit": "to fry with"}, {"id": 3, "name": "Chives or green onions", "quantity": null, "unit": null}, {"id": 4, "name": "Marjoram, fresh herbs", "quantity": null, "unit": "preferable"}], "steps": [{"number": 1, "description": "Turn the Anglerfish on its back. With a sharp and preferably short-bladed knife, make an incision from the anus to just between the two small fins under the head.", "preparation_time": null, "cooking_time": null, "waiting_time": null, "used_ingredients": [{"id": 1}]}, {"number": 2, "description": "Remove the organs by gently pulling and cutting the membrane that connects each to the rest of the body.", "preparation_time": null, "cooking_time": null, "waiting_time": null, "used_ingredients": [{"id": 1}]}, {"number": 3, "de

...

Error in parsing Failed to parse Recipe from completion {"name": "Chicken Thighs with New Potatoes, Bacon, and Lemon", "serving_size": 4, "ingredients": [{"id": 1, "name": "chicken thighs", "quantity": 8, "unit": "pieces"}, {"id": 2, "name": "new potatoes", "quantity": 500, "unit": "grams"}, {"id": 3, "name": "bacon lardon", "quantity": 100, "unit": "grams"}, {"id": 4, "name": "shallots", "quantity": 8, "unit": "pieces"}, {"id": 5, "name": "oregano sprigs", "quantity": 5, "unit": "pieces"}, {"id": 6, "name": "olive oil", "quantity": 2, "unit": "tablespoons"}, {"id": 7, "name": "lemons", "quantity": 2, "unit": "pieces"}, {"id": 8, "name": "chicken stock", "quantity": 200, "unit": "milliliters"}, {"id": 9, "name": "white wine", "quantity": 100, "unit": "milliliters"}], "steps": [{"number": 1, "description": "Preheat the oven to 220\u00b0C/200\u00b0C with fan/gas 7.", "preparation_time": null, "cooking_time": null, "waiting_time": null, "used_ingredients": []}, {"number": 2, "description"

Retrying langchain_mistralai.chat_models.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ReadTimeout: .


Error in parsing Invalid json output: Here is the recipe for "Spiced Chicken Bowls with Roasted Sweet Potatoes" in the required JSON format:

```json
{
  "name": "Spiced Chicken Bowls with Roasted Sweet Potatoes",
  "serving_size": 2,
  "ingredients": [
    {"id": 1, "name": "Chicken Breasts", "quantity": 2, "unit": "unit"},
    {"id": 2, "name": "Turkish Spice Blend", "quantity": 1, "unit": "tablespoon"},
    {"id": 3, "name": "Almonds", "quantity": 1, "unit": "ounce"},
    {"id": 4, "name": "Basmati Rice", "quantity": 3/4, "unit": "cup"},
    {"id": 5, "name": "Sweet Potato", "quantity": 12, "unit": "ounce"},
    {"id": 6, "name": "Onion", "quantity": 2, "unit": "ounce"},
    {"id": 7, "name": "Greek Yogurt", "quantity": 100, "unit": "ml"},
    {"id": 8, "name": "Parsley", "quantity": null, "unit": "unit"},
    {"id": 9, "name": "Lemon", "quantity": 1, "unit": "unit"},
    {"id": 10, "name": "Garlic Salt", "quantity": 1.5, "unit": "teaspoon"},
    {"id": 11, "name": "Chicken Broth", 

Retrying langchain_mistralai.chat_models.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ReadTimeout: .


Error in parsing Invalid json output: Here is the recipe for Potato Leek Soup in the requested JSON format:

```json
{
  "name": "Potato Leek Soup",
  "serving_size": 4,
  "ingredients": [
    {
      "id": 1,
      "name": "unsalted butter",
      "quantity": 3,
      "unit": "tablespoons"
    },
    {
      "id": 2,
      "name": "large leeks",
      "quantity": 2,
      "unit": "unit"
    },
    {
      "id": 3,
      "name": "chicken or veggie stock",
      "quantity": 1,
      "unit": "quart"
    },
    {
      "id": 4,
      "name": "waxy potatoes",
      "quantity": 11,
      "unit": "ounces"
    },
    {
      "id": 5,
      "name": "bay leaf",
      "quantity": 1,
      "unit": "unit"
    },
    {
      "id": 6,
      "name": "heavy cream",
      "quantity": 3/4,
      "unit": "cup"
    },
    {
      "id": 7,
      "name": "white pepper",
      "quantity": null,
      "unit": "unit"
    },
    {
      "id": 8,
      "name": "ground nutmeg",
      "quantity": null,
      "unit

In [17]:
with open("parsed_recipes_all_8x7b.json", "w") as f:
    json.dump(outputs, f, indent=4, ensure_ascii=False)